In [3]:
import sys
!{sys.executable} -m pip install pygam

     |████████████████████████████████| 1.8 MB 3.6 MB/s eta 0:00:01
     |████████████████████████████████| 829 kB 6.4 MB/s eta 0:00:01
     |████████████████████████████████| 30.9 MB 816 kB/s eta 0:00:01    |███████                         | 6.7 MB 7.2 MB/s eta 0:00:04     |███████████▊                    | 11.4 MB 7.2 MB/s eta 0:00:03     |██████████████▊                 | 14.2 MB 1.8 MB/s eta 0:00:10     |████████████████████            | 19.4 MB 4.4 MB/s eta 0:00:03     |█████████████████████▏          | 20.4 MB 4.4 MB/s eta 0:00:03
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491059 sha256=1295f7212025f8cee3ea6406d556e2b098bb582ddb8db662a30d711803c55d8f
  Stored in directory: /Users/leebardon/Library/Caches/pip/wheels/2f/a0/d3/4030d9f80e6b3be787f19fc911b8e7aa462986a40ab1e4bb94
Successfully built future
You should consider upgrading via the '/Users/leebardon/.pyenv/versions/3.9.2/bin/python3 -m pip install --upgrade pip' command.


In [4]:
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from pygam import LinearGAM, s, f

In [6]:
import matplotlib.pyplot as plt

def partial_dependency_plots(predictors, plankton_gams_dict, SAVE_PATH):
    for group_name, gam in plankton_gams_dict.items():
        plot_pdp(predictors, gam, group_name, SAVE_PATH)


def plot_pdp(predictors, gam, group_name, SAVE_PATH):
    y_label = "Biomass ($\mathregular{mmol\ C/m^3}$)"
    title = "Partial Dependency of Biomass to Predictors (1987-2008)"
    labels = [
        "$\mathregular{NO_3}$",
        "$\mathregular{PO_4}$",
        "Si",
        "Fe",
        "SSS",
        "SST",
        "PAR",
    ]
    filename = f"{group_name}_pdp.pdf"
    fig, axs = plt.subplots(1, 7, figsize=(18, 5))
    fig.tight_layout(pad=3.4)

    for i, ax in enumerate(axs):
        XX = gam.generate_X_grid(term=i)
        ax.plot(XX[:, i], gam.partial_dependence(term=i, X=XX))
        ax.plot(
            XX[:, i], gam.partial_dependence(term=i, X=XX, width=0.95)[1], c="r", ls="-"
        )
        if i == 0:
            ax.set_ylabel(y_label, fontsize=12)
        ax.set_xlabel(labels[i], fontsize=12)
        plt.suptitle(title, fontsize=18, y=1.1)

#         plt.savefig(
#             f"{SAVE_PATH}/{filename}",
#             format="pdf",
#             dpi=1200,
#         )

In [7]:
predictors = pd.read_pickle("/Users/leebardon/Development/stats_biogeo_2021/data/processed/training_sets/predictors/predictors_X_3586.pkl")
predictors_r = pd.read_pickle("/Users/leebardon/Development/stats_biogeo_2021/data/processed/training_sets/predictors/random_predictors_X_3586.pkl")
gams_dict = pd.read_pickle("/Users/leebardon/Development/stats_biogeo_2021/results/gams_output/fitted_models/from_measurements/gams_dict.pkl")
rand_gams_dict = pd.read_pickle("/Users/leebardon/Development/stats_biogeo_2021/results/gams_output/fitted_models/from_random/gams_r_dict.pkl")

In [10]:
def get_plankton(path):
    plankton_dict = {
        "proko": pd.read_pickle(f"{path}/proko.pkl"),
        "pico": pd.read_pickle(f"{path}/pico.pkl"),
        "cocco": pd.read_pickle(f"{path}/cocco.pkl"),
        "diazo": pd.read_pickle(f"{path}/diazo.pkl"),
        "diatom": pd.read_pickle(f"{path}/diatom.pkl"),
        "dino": pd.read_pickle(f"{path}/dino.pkl"),
        "zoo": pd.read_pickle(f"{path}/zoo.pkl"),
    }
    return plankton_dict


def get_predictors(path):
    predictors = pd.read_pickle(f"{path}/predictors_X_3586.pkl")
    predictors_random = pd.read_pickle(f"{path}/random_predictors_X_3586.pkl")
    return predictors, predictors_random


def set_min_vals(plankton):
    for group_name, biomass in plankton.items():
        biomass[biomass < 1.001e-5] = 1.001e-5
    return plankton


def fit_gams(plankton, predictors):
    gams_dict = {}
    for group_name, biomass in plankton.items():
        gams_dict[f"{group_name}"] = LinearGAM().fit(predictors, biomass)
    return gams_dict